# Focal Loss for Dense Object Detection

- __ 발표자 : 정지원__
- __ 발표일 : 2017. 12. 17(일)__

## Abstract

가장 높은 정확도를 보이는 Object detector들은 대부분 R-CNN과 같은, two-stage방식을 기반으로 한다. 여기서 classifier는 후보 오브젝트 위치의 sparse set에 적용된다. 이와 반대로, one-stage detectors들은 규칙적이고 dense한 샘플들로 detect를 하며, 더 빠르고 단순하지만 two-stage만큼의 정확도를 못 내고 있다. 이 논문에서는, 왜 그런지에 대해 얘기해본다. 우리는 극심한 foreground-background class imbalance 문제가 가장 주요한 원인이라는 것을 발견했다. 이러한 class 임밸런스 문제를 일반적인 cross-entropy loss를 *__down-weights the loss assigned to well-classified examples__*로 바꾸는 것을 제안한다. __Focal Loss__는 트레이닝의 초점을 sparse한 어려운 example(object)에 맞추며, 쉬우면서 많은 양의 negatives(background)에 압도되지 않도록 방지를 시켜준다. 이 loss의 효과를 평가하기 위해, **_RetinaNet_**이라는 simple한 dense detector를 만들어서 사용하였다. 결과적으로 focal loss로 학습된 모델은, one-stage 모델의 속도와 비슷하며, two-stage 모델을 뛰어넘는 정확도를 보였다.

![Image](./figures/2.png)

## Introduction

현재 SOTA detector들은 two-stage의 proposal-driven 메커니즘을 기반으로 한다. 많이 알려진 R-CNN의 경우, 첫 번째 스테이지에서 sparse한 object locations에 대한 후보군을 뽑아내며, 두 번째 스테이지에서는 각각의 후보 location을 CNN을 통해 예측하게 된다. 이러한 sequence한 장점을 통하여, COCO challenging에서 top accuracy들을 보이곤 했다.

two-stage detector의 성공에도 불구하고, 이런 질문이 나온다. 왜 one-stage는 비슷한 성능을 내지 못할까? One-stage detector들은 object의 locations, scales, aspect ratio에 regular하며 dense한 샘플링에 적용된다. YOLO나 SSD와 같은 최신 one-stage detectors들은 10~40%의 정확도를 보이면서 더 빠르게 동작한다.

FPN/Mask-RCNN/Faster R-CNN의 변형과 같은 더 복잡한 two-stage detector의 AP를 보여주는 one-stage object detector를 제시한다. 이 결과를 얻기 위해서는, 클래스 imbalance 문제로 SOTA 정확도가 나오는 것을 방해하는 문제를 새로운 loss를 통해 해결해야 한다.

Class imbalance는 R-CNN과 같은 detector에서는 two-stage cascade와 sampling heuristics을 통해 해결된다. Proposal stage는 후보 object locations 중, 대부분의 background samples을 필터링하여, 후보군을 빠르게 좁힌다. 두 번째 classification stage에서는, foreground와 background의 비율을 1:3과 같은 비율로 고정시키거나, OHEM(online hard example mining)등의 방법을 통해 균형을 맞춰주게 된다.

이와 다르게, one-stage detector는 이미지 전체에 걸쳐 일반적으로 샘플링되는 훨씬 많은 수의 후보군 locations을 처리해야 한다. 실제로, spatial positions, scales, aspect ratios등을 조밀하게 커버하는 100k개의 locations을 보여주는 경우가 많다. 유사하게 sampling heuristics가 적용되지만, 학습이 쉽게 분류되는 배경의 examples에 의해 지배되기 때문에 비효율적이다. 이러한 비효율은 일반적으로 Bootstrapping 또는 HEM(hard example mining)과 같은 기술을 통해 해결되는 object detection의 고전적인 문제다.

이 논문에서는, class imbalance 문제를 해결하기 위한 이전의 접근 방식과는 다른, 새로운 loss function을 제안한다. 이 loss functions은 동적으로 스케일되는 cross entropy loss인데, scaling factor decays는 올바른 class의 신뢰도가 높아짐에 따라 0에 가까워진다. 직관적으로, 이 스케일링 팩터는 학습 중 쉬운 예제(background)의 기여도를 자동으로 줄여주고, 어려운 예제(foreground)에 모델을 빠르게 집중시킬 수 있다. 실험을 통해 제시된 Focal loss는 Sampling Heuristic이나 HEM을 사용한 방법, 이전에 one-stage detector를 위해 사용되던 SOTA techniques들을 훨씬 능가하는 정확도를 얻어낼 수 있음을 보여준다. 마지막으로 Focal loss의 정확한 형태는 중요하지 않으며, 다른 인스턴스화도 비슷한 결과를 얻을 수 있음을 보여준다.

제안된 focal loss의 효과를 입증하기 위해, RetinaNet이라는 간단한 one-stage object detector를 설계했다. 입력이미지에서 객체 위치를 밀집하게 샘플링하기 때문에 이렇게 이름이 붙었다. 이 디자인은 효율적인 in-network feature pyramid와, anchor boxes의 사용이 특징이다. RetinaNet은 효율적이며 정확하다. The Best Model은 ResNet-101-FPN을 backbone으로 하며, COCO test-dev에서 5fps를 뽑아내며 39.1의 AP를 얻었다. 이는 이전 SOTA detector의 성능을 능가한다.

## Related Work

### Classic Object Detectors

Classifier를 dense image grid에 적용한 Sliding-window 패러다임은 길고 풍부한 역사가 있다. 최초의 성공 중 하나는 LeCun등의 Conv-net을 이용하여 Mnist를 맞추는 고전적인 작업이었다. HOG등의 도입... 들을 통해 수년 간 많은 연구가 있었다. 이러한 슬라이딩 윈도우 접근법은 고전적인 컴퓨터 비전의 선도적인 detector 패러다임이었지만, Deep learning이 나오면서 뒤이어 말할, two-stage detector가 뜨게 된다.

### Two-stage Detectors

요즘은 이러한 two-stage 기반의 detector가 많다. SS(Selective Search)가 sparse한 proposals의 후보군을 생성하면, 이것을 second stage에서 foreground/background로 분류한다. R-CNN은 second-stage를 CNN구조로 바꿔서, 정확도를 크게 향상시켰다. 몇 년간 R-CNN은 속도와 object proposal을 학습하는 방법을 통해 발전됐다. RPN을 포함하는 구조의 Faster R-CNN이 만들어졌으며, 이외에도 수많은 방법들이 제안됐다.

### One-stage Detectors

_OverFeat_은 깊은 네트워크를 기반으로 한 최초의 one-stage detector 중 하나였다. 최근, SSD와 YOLO등이 많이 사용되고 있으며, 이러한 디텍터들은 속도를 위해 조정됐기 때문에, 정확성은 two-stage보단 좋지 않았다. SSD는 10-20% lower AP, YOLO는 speed/accuracy의 trade-off을 절충하며 만들어졌다. 최근 연구 결과에 따르면, 이미지 해상도와 proposals 수를 줄임으로써 two-stage detector를 간단하게 만들 수 있지만, 계산을 더 많이해도 one-stage 방법으로는 정확도가 더 안좋게 나왔다. 이와 다르게, 이 연구에서는 one-stage와 유사하거나 더 빠른 속도를 내면서 two-stage의 정확도와 비슷하거나 더 높게 할 수 있을지에 대해 알아본다.

RetinaNet의 설계는 이전의 dense 감지기, 특히 RPN에 의해 도입된 'anchors'의 개념과 SSD 및 FPN에서와 같은 피라미드 feature의 사용과 많이 유사하다. 네트워크 구조의 혁신이 아닌, focal loss를 통해 top results를 냈음을 강조한다.

### Class Imbalance

Boosted Detector와 DPM과 같은 고전적인 one-stage object detecion과, SSD와 같은 최근의 방법은 모두 학습과정에서 큰 불균형에 직면해 있다. 이러한 detector들은 각 이미지당 10^4-10^5개의 후보를 가지지만, 실제로는 아주 적은 objects를 포함하고 있다. 이러한 불균형은 두 가지 문제를 일으킨다.

- 대부분의 위치(easy negatives)가 학습에 유용하지 못하다.
- easy negatives가 모델을 압도하여 생성해낼 수 있다.

일반적인 방법은 _hard negative mining_을 통하여 학습동안 hard examples을 샘플링하거나, 더욱 복잡한 sampling이나 reweighting schemes을 세우는 것이다. focal loss를 통해 one-stage detector가 갖는 imbalance 문제를 해결하고, easy negatives에 의해 overwhelming된 loss를 방지하면서 특별한 sampling이 없이 모든 examples에 대해 효율적으로 학습하게 된다.

### Robust Estimation

이전에는 큰 error가 있는 examples(hard examples)의 loss을 줄여서, __outliers__의 기여도를 줄이는 robust한 loss function을 설계하는 데 많은 관심들이 있었다. 이와 반대로, outliers를 다루기보다는, focal loss는 수가 많더라도 전체 손실에 대한 기여도가 작아지도록 inliers(쉬운 예)를 down-weighting함으로써 class imbalance를 해결하도록 설계했다. 즉, focal loss는 robust loss의 반대 역할을 수행한다. 즉, 어려운 예제의 sparse한 examples에 대해 학습을 집중시킨다.

## Focal Loss

Focal Loss는 foreground와 background calss들이 매우 imbalance한 상황에서, one-stage object detecion에 적용되기 위해 디자인 됐다. 우선, binary classification을 위한, __Cross Entropy__에 대해 소개한다.

![Image](./figures/12.png)

p는 다음과 같이 구해진다.

![Image](./figures/13.png)

따라서 $CE(p,y) = CE(p_{t}) = -log(p_{t})$ 로 표현할 수 있다.

![Image](./figures/1.png)

CE loss는 위 그림에서의 blue curve이다. 여기서 볼 수 있는 것은, 심지어 쉽게 분류된 examples에 대해서도 loss를 가지게 된다는 것이다. 이러한 많은 쉬운 예제들의 loss가 합쳐지면, 이것들이 rare class들을 overwhelm하게 된다.

### Balanced Cross Entropy

class imbalance를 해결하는 기본적인 방법은, 앞서 소개했던 weighting factor $\alpha$를 class 1에 곱하고, $1-\alpha$를 class -1에 곱하는 것이다. 실제로, $\alpha$는 incerse class 빈도에 의해 결정되거나, 혹은 cross validaion의 하이퍼파라미터로 취급될 수 있다. 우리는 $\alpha$-balanced CE loss를 다음과 같이 정의한다.

#### $CE(p_{t}) = -\alpha_{t} log(p_{t})$

이 loss는 CE의 simple extension이며, focal loss를 제안하는데 있어, baseline으로 고려하였다.

### Focal Loss Definition

앞서, large class imbalance가 cross entropy loss를 압도되는 문제를 보임을 말했었다. 쉽게 분류된 negatives들이 대부분의 loss를 구성하며 gradients도 지배한다. $\alpha$는 positive/negative 예의 밸런스를 잡는 반면에, 이것이 easy/hard한 것들을 구분하지는 못한다. loss function을 재구성하여, easy examples를 줄이고, hard negative에 집중하는 방식을 제안한다.

Cross entropy loss에 튜닝이 가능한 _focusing parameter_ $\gamma(>=0)$를 제안한다. 다음과 같이 정의한다.

#### $FL(p_{t}) = -(1-p_{t})^{\gamma}log(p_{t})$

$\gamma$=0이면 FL은 CE와 같다. $\gamma$가 증가함에 따라 변하는 모양을 보자. 실험 결과, $\gamma=2$일 때, 가장 좋은 성능을 보였다.

직관적으로, 쉬운 예제에서는 loss의 영향을 줄여주며, 낮은 loss를 받는 범위를 확장한다. 예를 들어, $\gamma=2$이고, $p_{t}=0.9$라면, CE와 비교할 때, 100배 작은 loss를 준다. 또한 $p_{t}=0.968$일 때는 1000배정도 작은 loss를 준다. 이로 인해, 잘 못 맞추는 예제들에 대해서는 상대적으로 중요성을 높여주는 효과가 난다.

$\alpha$-balanced variant를 적용하면 다음과 같다.

#### $FL(p_{t}) = -\alpha_{t}(1-p_{t})^{\gamma}log(p_{t})$

이렇게 적용한 결과, 실제 좀 더 좋은 성능을 보였다. 마지막으로, loss 연산을 통해 p값를 계산하기 위한 sigmoid 연산을 loss layer에 추가하였을 때, 수치가 안정적인 것을 확인하였다.

주요한 실험 결과에서 Focal loss definition을 사용하였지만, 정확한 형태는 중요하지 않았다. 부록에서 더 자세히 다루겠다.

### Class Imbalance and Model Initialization

Binary Classification 모델은 y=-1 or y=1일 확률이 같도록 초기화한다. 이러한 초기화는 실제 class imbalance한 상황에서는 빈번하게 등장하는 class의 loss가 total loss를 지배할 수 있으며, 초기 학습과정에서 불안정성을 보이게 된다. 이러한 문제를 해결하기 위해, 학습 시작시 rare한 class를 위해 모델에 의해 추정된 p값을 _prior_라는 개념을 통해 소개한다. prior를 $\pi$로 표현하고, rare class의 example에 대한 모델의 추정된 p가 낮도록, 0.01과 같은 값으로 설정한다. 이것은 초기화를 바꾸는 것이지, loss를 변경하는 것이 아니다. 이런 방법이 class imbalance가 심한 상황에서, CE나 FL에서 모두 학습을 안정적으로 한다는 것을 확인하였다.

### Class Imbalance and Two-stage Detectors

Two-stage detector는 대부분 $\alpha$-balancing이나 FL를 사용하지 않은 cross entropy loss를 사용하여 학습된다. 대신, class imbalance를 두 가지 메커니즘으로 해결한다.

- a two-stage cascade
- biased minibatch sampling

cascade stage의 경우, 거의 무한에 가까운 possible object locations을 1~2K개로 바꾸는 object propocal mechanism이다. 중요한 것은, 선택된 proposals은 랜덤하게 구해지는 것이 아니라, true object locations로부터 구해지기 때문에 아주 많은 easy negatives를 제거할 수 있다. Second-stage에서 biased sampling을 통해 minibatch 상에서 positive와 negative의 비율을 1:3의 비율로 보내는 등의 방법을 사용하게 된다. 이러한 ratio는 결국 $\alpha$-balancing과 유사하다. 이 논문에서 제안된 focall loss는 이러한 메커니즘의 아이디어를 loss function에 적용하여 one-stage에 적용한 것이라고 볼 수 있다.

## RetinaNet Detector

RetinaNet은 _backbone network_와 두 개의 task-specific _subnetworks_로 구성된 네트워크다. Backbone은 input 이미지에 대해 convolutional feature map을 계산하는 역할을 담당하는 Conv net이다. 첫 번째 subnet은 backbone의 출력에서 classification을 수행한다. 두 번째 subnet은 Bounding box regression을 수행한다. 두 subnetworks는 one-stage 고밀도 검출을 위해 특별히 만들어진 단순한 설계가 특징이다. 이러한 구성의 세부 사항에 대해 가능한 많은 옵션이 있지만, 대부분의 디자인 파라미터는 특별히 exact value에 민감하지 않았다. RetinaNet을 더 자세히 알아보자.

![Image](./figures/3.png)

### Feature Pyramid Network Backbone

RetinaNet의 Backbone으로 FPN을 사용하였다. 간단히 말하면, FPN은 일반적인 Convolutional network에 top-down pathway와 lateral connections을 갖는다. 그렇기 때문에, network는 효율적으로 rich, multi-scale feature pyramid를 single resolution image로부터 구성할 수 있다. 각 pyramid level은 서로 다른 scale의 object를 detect하는데 사용된다. FPN의 앞 부분을 ResNet 아키텍처로 사용하였다. level 3~7까지 pyramid를 구성하였다. 모든 피라미드 레벨은 C=256개의 채널을 갖는다. 더 구체적인 pyramid의 내용은 FPN 논문의 내용을 따르며, 살짝 수정하였다. 많은 디자인의 선택들은 중요하지 않으며, FPN을 backbone으로 사용한 것은 최종 ResNet 레이어의 feature만 사용하는 실험에서 낮은 AP를 나타냈기 때문이다.

### Anchors

FPN의 RPN 변종과 유사한, translation-invariant anchor boxes를 사용했다. 앵커는 피라미드 레벨 P3부터 P7까지 각각 32^2부터 512^2까지의 영역을 가지고 있다. FPN에서와 같이, {1:2, 1:1, 2:1}의 비율을 사용했다. FPN보다 밀도가 높은 커버리지의 경우, 각 레벨에서 3개의 앵커{2^0, 2^(1/3), 2^(2/3)} 크기를 추가하였다. 이를 통해 AP가 향상됐다. 각 레벨에서 총 A=9개의 앵커를 사용하였으며, 레벨에 따라 32-813 pixels의 이미지들을 커버할 수 있었다.

각 앵커는 클래스의 종류인 K의 길이를 갖는 one-hot vector와, 4-vector의 regresssion의 값이 할당된다. RPN로부터의 assignment rule을 사용하였으나, multi-class detection을 위해 조금 수정하였으며, threshold를 사용하였다. 특히, 앵커들은 GT boxes와의 IoU threshold를 0.5로 사용하였으며, [0, 0.4)는 background로 취급하였다. 각 앵커는 최대 one object box가 할당됐으며, 길이 K label vector 벡터에 정답=1, 나머지=0를 넣어서 학습했다. IoU가 [0.4, 0.5)인 경우는 anchor를 할당시키지 않았으며, 학습동안은 무시하였다. Box Regression은 각 앵커별로 계산을 하여, object box에 할당하거나, 할당이 안 된 경우 그냥 무시한다.

### Classification Subnet

Classification subnet은 각 A개의 앵커에 대해, K개의 오브젝트 클래스에 대한 확률을 predict한다. 이 subnet은 각 FPN의 레벨마다 작은 FCN를 붙였다. 이 subnet의 파라미터들은 모든 피라미드 레벨에서 공유된다. 이 디자인은 단순하다. 각 피라미드 레벨로부터 C개의 feature map 채널을 가져와서, 4개의 3x3 conv layer(with ReLU)를 적용하고, K\*A개의 3x3 conv layer를 붙였다. 마침내, sigmoid activations을 통해 KA개의 binary predictions을 각 spatial location마다 구해내게 된다. 대부분의 실험에서 C=256, A=9를 사용하였다.

RPN과는 반대로, object classification subnet은 깊으며, 오직 3x3 convs를 사용하고, box-regression subnet과는 파라미터를 공유하지 않는다. 이러한 구조를 갖는 것이, 하이퍼 파라미터를 수정하는 것보다 더 중요하다는 것을 알 수 있었다.

### Box Regression Subnet

위와 비슷하게, 각 피라미드 레벨마다 FCN을 붙였다. 각 spatial location마다 4A개의 linear outputs을 반환해준다는 점을 제외하고는 classifcation subnetwork와 동일하다. A개의 앵커들은 각각 4개의 값을 갖는데, GT에 대한 상대적인 offset 값을 예측한다. 가장 최신 연구와는 다르게, 더 작은 파라미터를 사용하는 클래스에 독립적인 regression을 사용하였으며, 효과가 같다는 것을 발견했다. classification subnet과 regression subnet은 공통적인 구조를 공유하지만, 별도의 파라미터를 사용한다.

## Inference and Training

### Inference

RetinaNet은 ResNet-FPN backbone에 FCN을 붙여 사용한다. inference는 단순히 image를 forward하는 과정이다. 속도를 올리기 위해, detector의 confidence를 0.05로 thresholding 한 후, FPN 레벨마다 최대 1k개의 최고 득점 예측에서 Regression을 수행했다. 모든 레벨에서의 top predictions은 merge 된 후, threshold 0.5를 갖는 non-maximum supression를 거쳐 final detections을 하게 된다.

### Focal Loss

$\gamma=2$가 가장 좋다는 것을 발견했다. 학습동안, focal loss는 각 이미지마다 all~100k개의 앵커에 적용이 된다. 휴리스틱한 샘플링(RPN)이나 hard example mining(OHEM, SSD)를 사용하여 적은 수의 앵커를 선택하는 것과는 차이가 있다. 한 이미지에 대한 모든 focal loss는 all~100k개의 앵커에 대해 계산이 되는데, GT에 지정된 앵커의 수로 정규화가 된다. total anchors가 아닌, assigned anchors의 수로 normalization을 수행했다. 대다수의 앵커들은 easy negatives이며 focal loss에서는 무시할 수 있는 loss 값을 주기 때문이다. 마지막으로, rare class를 위해 $\alpha$를 넣었는데, $\gamma$와 상호작용을 한다는 것을 확인하였다. 실험에서는 $\gamma=2, \alpha=5$가 최고의 성능을 보였다.

### Initialization

ResNet-50-FPN과 ResNet-101-FPN backbones을 통해 실험했다. 모두 ImageNet1k을 통해 pre-trained된 모델이다. FPN에 추가된 새로운 layer는 FPN논문과 같이 초기화했다. 마지막 하나를 제외한 conv-layers는 Gaussian weight with $\sigma$=0.01을 사용하였다. 마지막 layer는 bias initialization을 $b = -log((1-\pi)/\pi)$를 하였으며, 모든 실험에서 $\pi=0.01$을 사용하였다.

### Optimization

Retinanet은 SGD를 사용하였다. synchronized SGD를 통해, 8개의 GPUs를 사용하였으며, 각 minibatch별로 총 16개의 이미지를 사용하였다.(2 per GPU) 90k번의 itertations에 대해서는 lr=0.01, 60k에 대해 1/10, 80k에 대해 또 다시 1/10을 곱하여 사용하였다. horizontal flip, Weight decay 0.0001, momentum 0.9를 썼다. BB regression에서는 focal loss에 smooth $L_{1}$ loss를 더해서 사용하였다. 10~35시간 정도가 걸렸다.

## Experiments

COCO trainval35k split을 사용하여 실험하였다. test-dev split에 대한 AP를 evaluation server로부터 평가받았다.

![Image](./figures/4.png)

## Training Dense Detection

여러 종류의 optimization 전략을 사용하면서 dense detection에 대해 loss functions이 어떻게 동작하는지 분석하기 위해 많은 실험을 하였다.

### Network Initialization

우선 CE를 사용하여 아무것도 변경하지 않은 채로 실험을 하였다. 모델이 자꾸 발산하여 실패하였다. 그러나 마지막 layer를 prior probability of detecting object $\pi = 0.01$ 로 초기화를 하여, 이 문제를 쉽게 해결하였다. 이미 ResNet-50을 사용한 RetinaNet은 AP 30.2를 달성했다.

### Balanced Cross Entropy

$\alpha$-balanced CE loss를 통해 어떻게 개선이 되는지 확인해본다. $\alpha=0.75$를 사용하여, 0.9의 AP를 높였다.

![Image](./figures/6.png)

### Focal Loss

결과는 위 테이블에서 보여주고 있다. $\gamma=0$이면 CE와 같다. $\gamma$가 증가할수록, easy example에 대한 low loss는 잘 고려하지 않게 된다. $\alpha$-balanced CE loss에 비해, $\gamma=2$인 FL를 사용하여, 2.9AP를 향상시켰다. $\alpha$와 $\gamma$를 변경하며 많은 실험을 하였으며, 결국 $\gamma=2.0, \alpha=0.25$가 가장 좋은 결과를 보였다.

### Analysis of the Focal Loss

focal loss가 더 좋다는 것을 이해하기 위해서, 수렴하는 모델의 loss의 empirical 분포를 분석하였다. 이를 위해, default ResNet 101 600-pixel model을 $\gamma=2$로 학습하였다.(36.0 AP) 이 모델에 아주 많은 random-images를 넣었으며, 이것들은 약 10^7개의 negative windows와 ~10^5개의 positive windows를 생성하였다. 그리고서, 각 positives와 negatives에 대해 FL를 구하였고, normalize한 loss를 sum하였다. 구해진 normalized loss를 lowest부터 highest까지 정렬하여 각 positives와 negatives sample에 대해 CDF를 그려보았다.($\gamma$를 바꿔가면서, 모델은 $\gamma=2$로 학습을 시켰지만.)

![Image](./figures/5.png)

positives와 negatives에 대한 CDF는 위와 같다. positive samples의 경우, CDF는 $\gamma$값에 큰 영향을 안 받는 것으로 보인다. 약 20%의 hardest positive samples들은 대략 반 정도의 positive loss를 보이며, $\gamma$가 증가함에 따라서 top 20%의 examples에 집중하는 경향을 보였지만, 효과는 minor하다.

negative samples에 대한 $\gamma$의 효과는 매우 다르다. $\gamma=0$의 경우 CDF는 거의 비슷하다. 그러나, $\gamma$가 증가함에 따라, 실질적으로 더 많은 weights가 hard negative examples에 집중한다. 실제로, $\gamma=2$의 경우, 대부분의 loss는 sample의 작은 부분에서 비롯된다. 보다시피, FL는 hard negative에 주의를 집중하면서 easy negative를 효과적으로 discount할 수 있다.

### Online Hard Example Mining(OHEM)

minibatchs를 high-loss examples을 통해 재구성 함으로써, two-stage detector를 향상시키기 위해 제안됐다. 특히 OHEM에서, 각 example은 loss에 의해 스코어링 되며, non-maximum suppression이 적용됐으며, minibatch는 highest-loss examples로 구성됐다. Non-maximum suppression(nms) threshold와 batch size는 튜닝가능한 파라미터다. focal loss와 같이, OHEM는 misclassified examples을 더욱 강조한다. 하지만 FL와는 다르게, easy examples를 완벽히 버리게 된다. OHEM는 SSD에서 사용됐다. 모든 examples에 nms를 적용한 후에, minibatch가 1:3의 positive:nagative 비율로 구성되며, 각 minibatch는 충분한 positives를 갖게 된다.

OHEM를 one-stage에 실험해봤다. Original OHEM 전략과, OHEM 1:3 전략의 결과가 위 표에 나와있다. FL를 넣은 ResNet-101이 36.0 AP를 보인 반면에, OHEM을 통해서는 32.8AP를 얻었다. 따라서, FL이 더욱 효과적임을 보여준다. OHEM을 여러 파라미터를 바꿔가며 실험을 해봤지만, 더 나은 결과를 얻을 수는 없었다.

### Hinge Loss

마침내, hinge loss를 $p_{t}$에 적용하려는 시도를 하였다. 그러나, 이것은 불안정했고 의미있는 결과를 얻을 수 없었다. 결과는 부록에 있다.

## Model Architecture Design

### Anchor Density

아키텍처 디자인 요소 중 가장 중요한 것은 one-stage detection system은 가능한 이미지 박스들의 공간을 얼마나 densely 커버할 수 있는가이다. Two-stage detectors는 region pooling operation을 통해 어떠한 position, scale, aspect ratio의 boxes에 대해서도 classify할 수 있다. 이와 대조적으로, one-stage detectors는 fixed sampling grid를 사용하기 때문에, 이러한 접근법에서 boxes의 high coverage를 달성하기 위한 보편적인 접근법은 다양한 scale, aspect ratio의 box를 커버하기 위해 각 position에 여러 Anchor를 사용하는 것이다.

FPN에서 각 position과 각 피라미드 레벨에서 사용되는 scale 및 aspect-ratio 앵커의 수를 살펴보자. 각 위치의 단일 정사각형 앵커에서 4개의 sub-octave scales($2^{k/4}$, for k<=3)와 3 aspect ratio[0.5, 1, 2]에 걸쳐 position당 12개의 앵커를 사용한다. ResNet-50을 사용한 결과는 위 테이블에 있다. 놀랍게도 하나의 Square 앵커를 사용했을 때도 결과가 좋았다. 그러나, 3scales과 3aspect ratios를 사용하여 약 4AP를 향상시켰다. 따라서 모든 실험에서는 이를 고정하여 사용하였다.

더 많은 앵커를 사용해봤자 효과가 없다는 것을 확인하였다. 따라서 two-stage 시스템은 이미지의 임의의 상자를 분류할 수 있지만, 밀도와 관련된 성능의 포화로, two-stage 시스템의 잠재적인 밀도가 높을수록 이점을 제공하지 못 할 수 있음을 의미한다.

### Speed versus Accuracy

더 큰 backbone networks는 높은 정확도를 주지만, inference 속도가 느려진다. Input image scale도 마찬가지다. 그 효과에 대해 위 표에 나타냈다. RetinaNet with ResNet-101-FPN은, ResNet-101-FPN Faster R-CNN과 비슷한 성능을 보이며, 122ms per image의 속도를 보여줬다.(vs 172ms) 더 큰 이미지를 사용했을 때는, RetinaNet은 모든 two-stage 방식의 accuracy를 능가하였으며, 심지어 더 빠르다. 아주 빠른 프레임 속도를 보이는 것은, 특별한 네트워크 설계를 요구할 가능성이 있으므로 범위를 벗어나 다루지 않는다.

## Comparison to State of the Art

COCO 대회, test-dev를 제출하여 SOTA 결과를 얻었다. one-stage/two-stage 모두에서 더 좋은 결과를 얻었다.

## Conclusion

이 논문에서는, One-stage object detector가 FAST R-CNN의 변형과 같은 최고 성능의 two-stage을 능가하지 못하게 하는 주요 장애물을 Class imbalance로 여겼다. 이를 해결하기 위해 Hard-examples을 학습하고 수많은 easy negatives를 줄이기 위해 Cross entropy loss에 modulating term을 적용하는 __focal loss__를 제안했다. 이러한 접근 방식은 간단하고 매우 효과적이다. 완벽한 Convolutional one-stage detector를 설계하여 효과을 입증하고 까다로운 COCO 데이터셋에서 SOTA 정확도와 실행 시간을 달성한다는 광범위한 실험 분석을 보고했다.

---

## Appendix A: Focal Loss*

Focal loss의 정확한 형태는 중요하지 않다. focal loss와 유사한 성질을 갖고 비슷한 결과를 보여주는 대체 instantiation을 소개한다. 다음은 focal loss의 속성에 대한 더 많은 통찰력을 제공한다.

Cross Entropy와 Focal Loss를 이전에 소개한 것과 약간 다른 형태로 고려하여 시작한다. 다음과 같이 quantity $x_{t}$를 정의한다.

#### $x_{t} = yx$

여기서 y는 이전에 말한 것과 같이, ground-truch class를 특정한다. 이제, $p_{t} = \sigma(x_{t})$로 쓸 수 있다. 예를 들어, $p_{t}>0.5이며 $$x_{t}>0$. focal loss의 alternate form을 다음과 같이 쓸 수 있다.

#### - ${p_{t}}^{*} = \sigma(\gamma x_{t} + \beta)$
#### - $FL^{*} = -log({p_{t}}^{*}) / \gamma$

FL\*는 두 파라미터, $\gamma$, $\beta$를 갖는데, loss curve의 steepness와 shift를 결정한다. 

![Image](./figures/7.png)
![Image](./figures/8.png)
![Image](./figures/9.png)
![Image](./figures/10.png)
![Image](./figures/11.png)

# Code

## loss functions

In [ ]:
from keras import backend as K

def categorical_crossentropy(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)

In [ ]:
def focal_loss(y_true, y_pred, gamma=2, alpha=2):
    import tensorflow as tf
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    return -K.sum(alpha * K.pow(1. - pt, gamma) * K.log(pt))